**Algorithm for ChiMerge**
    1. Take the distinct set of values and sort it in ascending order
    2. For all the distinct values assign a interval
    3. While the humber of intervals greater than maximum interval allowed we have repeat below steps
        a. Start the for loop starting i from 0 to number of intervals-1
            i. calculate observed and expected values np array of i and i+1 interval (it says adjacent interval)
            ii. Calculate the chi square between the two adjacent intervals
            iii. append that chi square value to the chi list containing all the chi square values of adjacent intervals
        b. Find the min chi square value index
        c. start a for loop starting from i =0 to number of intervals
            i. if i is the min_index, you merge the two intervals i and i+1. Skip the i+1 interval as its already merged.
            ii. Add all the old intervals to new intervals list
        d. Reset intervals with new intervals so that its again ready for next iteration

In [1]:
import pandas as pd
from collections import Counter
import numpy as np

In [3]:
iris = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)


In [4]:
iris.columns = ['sepal_l', 'sepal_w', 'petal_l', 'petal_w', 'type']

In [16]:
iris.shape

(150, 5)

In [5]:
iris.head()

,sepal_l,sepal_w,petal_l,petal_w,type
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [23]:
def create_new_interval_with_merging(old_interval,min_chi_index):
    new_intervals = [] # Prepare for the merged new data array
    skip = False
    done = False
    for i in range(len(old_interval)):
        if skip:
            skip = False
            continue
        if i == min_chi_index and not done: # Merge the intervals
            t = old_interval[i] + old_interval[i+1]
            new_intervals.append([min(t), max(t)])
            skip = True
            done = True
        else:
            new_intervals.append(old_interval[i])
    
    return new_intervals


In [38]:
def calc_chi_square(i,data,attr,intervals,empty_count,label):
    obs0 = data[data[attr].between(intervals[i][0], intervals[i][1])]
    obs1 = data[data[attr].between(intervals[i+1][0], intervals[i+1][1])]
    total = len(obs0) + len(obs1)
    count_0 = np.array([v for i, v in {**empty_count, **Counter(obs0[label])}.items()])
    count_1 = np.array([v for i, v in {**empty_count, **Counter(obs1[label])}.items()])
    count_total = count_0 + count_1
    expected_0 = count_total*sum(count_0)/total
    expected_1 = count_total*sum(count_1)/total
    chi_ = (count_0 - expected_0)**2/expected_0 + (count_1 - expected_1)**2/expected_1
    chi_ = np.nan_to_num(chi_) # Deal with the zero counts
    return sum(chi_)
    

In [39]:
def chimerge(data, attr, label, max_intervals):
    distinct_vals = sorted(set(data[attr])) # Sort the distinct values
    labels = sorted(set(data[label])) # Get all possible labels
    empty_count = {l: 0 for l in labels} # A helper function for padding the Counter()
    intervals = [[distinct_vals[i], distinct_vals[i]] for i in range(len(distinct_vals))] # Initialize the intervals for each attribute
    while len(intervals) > max_intervals: # While loop
        chi = []
        for i in range(len(intervals)-1):
            # Calculate the Chi2 value by calling calc_chi_square
            chi_value=calc_chi_square(i,data,attr,intervals,empty_count,label)
            chi.append(chi_value) # Finally do the summation for Chi2
        min_chi = min(chi) # Find the minimal Chi2 for current iteration
        for i, v in enumerate(chi):
            if v == min_chi:
                min_chi_index = i # Find the index of the interval to be merged
                break
        
        #Call create_new_interval_with_merging to merge the least chi square ajacent values and create new interval
        intervals = create_new_interval_with_merging(intervals,min_chi_index)
        
    
    return intervals
        


In [40]:
for attr in ['sepal_l', 'sepal_w', 'petal_l', 'petal_w']:
    print('Interval for', attr)
    intervals=chimerge(data=iris, attr=attr, label='type', max_intervals=6)
    
    print("Split points are as below : ")
    for i in intervals:
        print (i[0])
    
    print("\n Below are the intervals : ")
    for i in intervals:
        print('[', i[0], ',', i[1], ']', sep='')

Interval for sepal_l


C:\Users\akash\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


Split points are as below : 
4.3
4.9
5.0
5.5
5.8
7.1

 Below are the intervals : 
[4.3,4.8]
[4.9,4.9]
[5.0,5.4]
[5.5,5.7]
[5.8,7.0]
[7.1,7.9]
Interval for sepal_w
Split points are as below : 
2.0
2.3
2.5
2.9
3.0
3.4

 Below are the intervals : 
[2.0,2.2]
[2.3,2.4]
[2.5,2.8]
[2.9,2.9]
[3.0,3.3]
[3.4,4.4]
Interval for petal_l
Split points are as below : 
1.0
3.0
4.5
4.8
5.0
5.2

 Below are the intervals : 
[1.0,1.9]
[3.0,4.4]
[4.5,4.7]
[4.8,4.9]
[5.0,5.1]
[5.2,6.9]
Interval for petal_w
Split points are as below : 
0.1
1.0
1.4
1.7
1.8
1.9

 Below are the intervals : 
[0.1,0.6]
[1.0,1.3]
[1.4,1.6]
[1.7,1.7]
[1.8,1.8]
[1.9,2.5]
